In [1]:
from numpy import std, arange
from random import uniform
from math import exp
from bokeh.models import ColumnDataSource, Span
from bokeh.plotting import figure, output_file, save, show, curdoc
from bokeh.io import output_notebook, reset_output


In [2]:
# Set parameter
TEMPLATE_FILE_NAME = "clean_lc"
FLARE_HEIGHT = 3
# Set number of frames
FRAMES_RISE = 10
FRAMES_DECAY = 110
# Set starting and ending point of flare
STARTING_POINT = 200
ENDING_POINT = STARTING_POINT + FRAMES_RISE + FRAMES_DECAY - 1

# Spanning the time region −1 < t1/2 ≤ 0 for rising phase
# Peak time must be zero (t1/)
# 3 < t1/2 < 6 for time region of decay
TIME_PEAK = 0
TIME_RISE = 1
TIME_DECAY = 5

In [3]:
# REF : James R. A. Davenport et al 2014 ApJ 797 122
# Kepler Flares paper II Eq.1 -1 < t < =0
def kepler_rise_phase(t):
    a0 = 1
    a1 = (1.941 + uniform(-0.008, 0.008)) * t
    a2 = (-0.175 + uniform(-0.032, 0.032)) * t * t
    a3 = (-2.246 + uniform(-0.039, 0.039)) * t * t * t
    a4 = (-1.125 + uniform(-0.016, 0.016)) * t * t * t * t
    f_rise = a0 + a1 + a2 + a3 + a4
    return f_rise


# Kepler Flares paper II Eq.4 0 < t < n
def kepler_decay_phase(t):
    fast_phase = (0.6890 + uniform(-0.0008, 0.0008)) * exp((-1.6 + uniform(-0.003, 0.003)) * t)
    slow_phase = (0.3030 + uniform(-0.0009, 0.0009)) * exp((-0.2783 + uniform(-0.0007, 0.0007)) * t)
    return fast_phase + slow_phase

In [4]:
# Loading sample data
sample_data = []
raw_data = []
with open(TEMPLATE_FILE_NAME, "r") as sample_file:
    for line in sample_file:
        sample_data.append(float(line))
        raw_data.append(float(line))

In [5]:
# Check indexes of flare that do not have more than the length of the input file.
if ENDING_POINT >= len(sample_data):
    raise IndexError("Index of artifact flare out of bounds")

In [6]:
# Calculate standard deviation (sigma) of sample data
sigma = std(sample_data)

# Create time scales for rising and decay phases
timescale_rising_phase = arange(-TIME_RISE, TIME_PEAK, TIME_RISE / FRAMES_RISE).tolist()
timescale_decay_phase = arange(TIME_PEAK, TIME_DECAY, TIME_DECAY / FRAMES_DECAY).tolist()
timescale_list = timescale_rising_phase + timescale_decay_phase


In [7]:
# Inject Kepler flare
for index, time_scale in enumerate(timescale_list):
    # Check if time scale is in the rising phase
    if (-TIME_RISE <= time_scale) and (time_scale <= TIME_PEAK):
        flux_flare = kepler_rise_phase(t=time_scale)
    # If not, it's in the decay phase
    else:
        flux_flare = kepler_decay_phase(t=time_scale)
        # Calculate flux flare and add it to the sample data
    flux_flare = flux_flare * (FLARE_HEIGHT * sigma)
    sample_data[STARTING_POINT + index] = sample_data[STARTING_POINT + index] + flux_flare


In [8]:
# # Set the output file for the plot
# output_file(filename="result.html")

# If you want to display Bokeh plot in Jupyter notebook, please use output_notebook()
# instead of output_file() in line 2
reset_output()
output_notebook()

# Set the tooltips for the plot
TOOLTIPS = [
    ("index", "$index"),
    ("(x, y)", "$x, $y"),
]

# Create the x-axis data for the plot
x_axis_data = [*range(len(sample_data))]

# # Create the figure
# plot = figure(tooltips=TOOLTIPS, title="Result", x_axis_label="Time", y_axis_label="Flux", sizing_mode='stretch_both')

# If you want to display Bokeh plot in Jupyter notebook, please use line 22-23
# instead of line 18
plot = figure(tooltips=TOOLTIPS, title="Result", x_axis_label="Time", y_axis_label="Flux", width=600, height=400)

# Plotting raw data
raw_datasource = ColumnDataSource(data=dict(x=x_axis_data, y=raw_data))
plot.line('x', 'y', source=raw_datasource, line_alpha=0.5, color="black", line_width=2, legend_label="Raw")
plot.circle('x', 'y', source=raw_datasource, color="black", fill_alpha=0.6, legend_label="Raw", size=2)

# Plotting kepler data
kepler_datasource = ColumnDataSource(data=dict(x=x_axis_data, y=sample_data))
plot.line('x', 'y', source=kepler_datasource, line_alpha=0.5, color="blue", line_width=2, legend_label="Kepler")
plot.circle('x', 'y', source=kepler_datasource, color="blue", fill_alpha=0.6, legend_label="Kepler", size=2)

# Plotting transient pattern using Span
L_span = Span(location=STARTING_POINT, dimension='height', line_color="#C70039", line_dash='4 4', line_width=2)
R_span = Span(location=ENDING_POINT, dimension='height', line_color="#C70039", line_dash='4 4', line_width=2)
plot.add_layout(L_span)
plot.add_layout(R_span)

# Setting legend properties
plot.legend.border_line_width = 3
plot.legend.click_policy = "hide"
plot.legend.border_line_color = "navy"
plot.legend.border_line_alpha = 0.0

# # Save the plot
# save(plot)

# If you want to display Bokeh plot in Jupyter notebook, please use show(plot)
# instead of save(plot) in line 43
show(plot)

Loading BokehJS ...

In [9]:
# Export LC into txt file
FRAME_DURATION = FRAMES_RISE + FRAMES_DECAY
PEAK_POINT = STARTING_POINT + FRAMES_RISE
EXPORT_FILE_NAME = "{}_H{}_D{}_{}_{}".format(TEMPLATE_FILE_NAME, FLARE_HEIGHT, FRAME_DURATION, STARTING_POINT, PEAK_POINT)

with open(EXPORT_FILE_NAME, 'w') as output:
    for row in sample_data:
        output.write(str(row) + '\n')

print("Exporting {} completed".format(EXPORT_FILE_NAME))

Exporting clean_lc_H3_D120_200_210 completed
